<a href="https://colab.research.google.com/github/animesharma3/Pytorch-Code-Snippets/blob/main/Pytorch_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Imports
* Create Fully Connected Layer
* Set device
* Hyperparameters
* Load Data
* Initialize Network
* Loss and Optimizer
* Train Network
* Check Accuracy on training and test to see how good our model is

# Imports

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim # Optimizers
import torch.nn.functional as F # Activation functions - ReLu etc
from torch.utils.data import DataLoader # easier dataset mgmt -> creates mini batches to train on
import torchvision.datasets as datasets
import torchvision.transforms as transforms # transformations to perform on datasets

# Create Fully Connected Network

In [3]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes): # MNIST - (28, 28) -> 728
        super(NN, self).__init__() # super(type, obj)
        # The super() function is used to give access to methods and properties of a parent or sibling class.
        # The super() function returns an object that represents the parent class.
        self.fc1 = nn.Linear(in_features=input_size, out_features=50)
        self.fc2 = nn.Linear(in_features=50, out_features=num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
model = NN(784, 10)
x = torch.randn(64, 784)
print(model(x).shape) # Callable object
print(model.forward(x).shape)

torch.Size([64, 10])
torch.Size([64, 10])


In [8]:
scores = model(x)
scores.max()

tensor(0.9647, grad_fn=<MaxBackward1>)

In [40]:
print(x.shape)
print(model.fc1(x).shape) # Linear(64, 50) <- (64, 784)
print(model.fc2(model.fc1(x)).shape) # Linear(64, 10) <- (64, 50)

torch.Size([64, 784])
torch.Size([64, 50])
torch.Size([64, 10])


# Set Device

In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters

In [42]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1

# Load Data

In [43]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [54]:
print(train_dataset.train_data[0].shape)

torch.Size([28, 28])


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


# Initialize Network

In [44]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

# Loss and Optimzer

In [45]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [46]:
for para in model.parameters():
    print(para.shape)

torch.Size([50, 784])
torch.Size([50])
torch.Size([10, 50])
torch.Size([10])


# Train Network

In [47]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        ## Get Data to cuda
        data = data.to(device=device)
        targets = targets.to(device=device)
        # print(data.shape) -> torch.Size([64, 1, 28, 28])

        ## Get to correct shape
        data = data.reshape(data.shape[0], -1) # unravel the data -> [64, 728]

        # Forward
        scores = model(data)
        loss = criterion(scores, targets)

        # Backward
        optimizer.zero_grad()
        loss.backward()

        # Gradient Descent or Adam step
        optimizer.step()

In [48]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Training Data")
    else:
        print('Testing Data')

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        acc = float(num_correct)/float(num_samples) * 100
        print(acc)

    model.train()
    return acc
print(check_accuracy(train_loader, model))
print(check_accuracy(test_loader, model))

Training Data
93.0
93.0
Testing Data
93.13
93.13
